In [1]:
#@title Install and import necessary libraries
%pip install -qU tool-parse[langchain] langchain-ollama langgraph duckduckgo-search

from tool_parse.integrations.langchain import ExtendedStructuredTool, patch_chat_model

import typing as t
from duckduckgo_search import DDGS
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_ollama.chat_models import ChatOllama
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autoform 0.1.0 requires langchain-core<0.3.0,>=0.2.1, but you have langchain-core 0.3.5 which is incompatible.
autoform 0.1.0 requires pydantic==2.6.4, but you have pydantic 2.9.2 which is incompatible.
chromadb 0.4.7 requires fastapi<0.100.0,>=0.95.2, but you have fastapi 0.110.1 which is incompatible.
chromadb 0.4.7 requires pydantic<2.0,>=1.9, but you have pydantic 2.9.2 which is incompatible.
instructor 0.6.2 requires docstring-parser<0.16,>=0.15, but you have docstring-parser 0.16 which is incompatible.
instructor 0.6.2 requires typer<0.10.0,>=0.9.0, but you have typer 0.12.3 which is incompatible.
langchain-anthropic 0.1.11 requires langchain-core<0.2.0,>=0.1.43, but you have langchain-core 0.3.5 which is incompatible.
langchain-aws 0.1.3 requires langchain-core<0.2.0,>=0.1.45, but you have langchain-core 0.

Note: you may need to restart the kernel to use updated packages.


#### Define tools

In [3]:
async def search_text(
    text: str,
    *,
    safe_search: bool = True,
    backend: t.Literal["api", "html", "lite"] = "api",
    max_results: int = 1,
):
    """
    Search for text in the web.
    :param text: Text to search for.
    :param safe_search: If True, enable safe search.
    :param backend: Backend to use for retrieving results.
    :param max_results: Max results to return.
    """
    return DDGS().text(
        keywords=text,
        safesearch="on" if safe_search else "off",
        backend=backend,
        max_results=max_results,
    )

class ProductInfo(t.NamedTuple):  # Can be t.TypedDict or pydantic.BaseModel
    """
    Information about the product.
    :param name: Name of the product.
    :param price: Price of the product.
    :param in_stock: If the product is in stock.
    """

    name: str
    price: float
    in_stock: bool = False

tools = [
    ExtendedStructuredTool(func=search_text),
    ExtendedStructuredTool(func=ProductInfo, name="product_info"),
]
# OR # tools = ExtendedStructuredTool.from_objects(search_text, ProductInfo)
tools

[ExtendedStructuredTool(name='search_text', description='Search for text in the web.', func=<function search_text at 0x000001840CC5F910>),
 ExtendedStructuredTool(name='product_info', description='Information about the product.', func=<class '__main__.ProductInfo'>)]

#### Patch the langchain chat model

In [4]:
model = patch_chat_model(ChatOllama(model="llama3-groq-tool-use")) # Patch the instance
# OR
model = patch_chat_model(ChatOllama)(model="llama3-groq-tool-use") # Patch the class and then instantiate it

#### Create a react agent

In [11]:
agent_executor = create_react_agent(model, tools, checkpointer=MemorySaver())
config = {"configurable": {"thread_id": "abc123"}}

def query_agent(__query: str):
    for chunk in agent_executor.stream({"messages": [HumanMessage(content=__query)]}, config):
        message = list(chunk.values())[0]["messages"][0]
        if isinstance(message, AIMessage):
            print("[AGENT]")
            if message.tool_calls:
                print("Calling tool:")
                metadata = message.tool_calls[0]
                print(f"name={metadata['name']!r}")
                print(f"arguments={metadata['args']}")
            else:
                print(message.content)
        elif isinstance(message, ToolMessage):
            print(f"[{message.name!r} TOOL OUTPUT]")
            print(message.content)

        print("---" * 20)

In [12]:
query_agent("Search 5 sources for langgraph docs using lite backend")

[AGENT]
Calling tool:
name='search_text'
arguments={'backend': 'lite', 'max_results': 5, 'text': 'langraph docs'}
------------------------------------------------------------
['search_text' TOOL OUTPUT]
[{"title": "LangGraph - LangChain", "href": "https://www.langchain.com/langgraph", "body": "LangGraph is a framework for building stateful, multi-actor agents with LLMs that can handle complex scenarios and collaborate with humans. LangGraph Cloud is a service for deploying and scaling LangGraph applications, with a built-in Studio for prototyping, debugging, and sharing."}, {"title": "️LangGraph - GitHub Pages", "href": "https://langchain-ai.github.io/langgraph/", "body": "LangGraph is a framework for creating stateful, multi-actor applications with LLMs, inspired by Pregel and Apache Beam. It offers cycles, controllability, persistence, human-in-the-loop, and streaming features, and integrates with LangChain and LangSmith."}, {"title": "langgraph/README.md at main · langchain-ai/langg

In [14]:
query_agent("Parse the product: RTX 4900, priced at $3.5k, is in stock.")

[AGENT]
Calling tool:
name='product_info'
arguments={'in_stock': True, 'name': 'RTX 4900', 'price': 3500}
------------------------------------------------------------
['product_info' TOOL OUTPUT]
["RTX 4900", 3500.0, true]
------------------------------------------------------------
[AGENT]
The product RTX 4900 is priced at $3,500 and is currently in stock.
------------------------------------------------------------
